In [2]:
from itertools import combinations
from Logica import *
#from Logica import Otoria, Ytoria, Descriptor, inorder_to_tree
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.offsetbox import AnnotationBbox, OffsetImage
from types import MethodType
from Reglas import *

r = Rejilla(X=3, Y=3)
print(r.CM.ravel([4, 0, 2]))

numero_8 = r.CM.ravel([7, 0, 0])
numero_3 = r.CM.ravel([2, 0, 1])
numero_4 = r.CM.ravel([3, 0, 2])
numero_1 = r.CM.ravel([0, 1, 0])
numero_5 = r.CM.ravel([4, 1, 1])
numero_9 = r.CM.ravel([8, 1, 2])
numero_6 = r.CM.ravel([5, 2, 0])
numero_7 = r.CM.ravel([6, 2, 1])
numero_2 = r.CM.ravel([1, 2, 2])
I = {
    numero_8 : True,
    numero_3 : True,
    numero_4 : True,
    numero_1 : True,
    numero_5 : True,
    numero_9 : True,
    numero_6 : True,
    numero_7 : True,
    numero_2 : True
}
r.visualizar(I)


#0, 2   1, 2   2, 2
#0, 1   1, 1   2, 1
#0, 0   1, 0   2, 0

ModuleNotFoundError: No module named 'Reglas'